In [1]:
import re, os, zlib, time, pickle, urllib, datetime, multiprocessing 

r = urllib.request.urlopen('ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/xml/')
ls_long = r.fp.read().decode() 
pdbids = [i.split()[8][:4] for i in ls_long.split('\r\n') if i]
# executed in 2m 41s, finished 09:18:06 2020-12-09

In [ ]:
re_dbver = re.compile(b'(?<=dbEntryVersion=\").*?(?=\")')
def sifts_get_database_version_by_pdbid(pdbid): 
    try:
        r     = urllib.request.urlopen(f'ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/xml/{pdbid}.xml.gz')
        gz    = r.fp.read(500) 
        xml   = zlib.decompressobj(32+zlib.MAX_WBITS).decompress(gz)
        ver   = re.search(re_dbver, xml).group(0).decode() 
        return pdbid, ver
    except:
        return None 

num_worker = 5
with multiprocessing.Pool(num_worker) as p, open('sifts.progress', 'w') as fw: 
    num_jobs = len(pdbids)
    jobs = [p.apply_async(sifts_get_database_version_by_pdbid, (i,)) for i in pdbids]
    while True:
        time.sleep(60) 
        num_ready = len([job.ready() for job in jobs if job.ready()])
        print(f'Finish rate: {num_ready} / {num_jobs} @ {datetime.datetime.now().isoformat()}', file=fw)
        fw.flush()
        if num_ready == num_jobs:
            result = [job.get() for job in jobs]
            with open('sifts.pkl', 'wb') as _fw: pickle.dump(result, file=_fw)
            break

#import concurrent 
#with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
#    executor.map(sifts_get_database_version_by_pdbid, pdbids) 